# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f565f02f8a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Trademarks across the world") 

Consider the following exploratory information need:

> You are investigating different kinds of trademarks

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wd:Q167270`  | trademark     | node      |
| `wd:Q14091`   | iMac          | node      |
| `wd:Q312`     | Apple Inc.    | node      |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q167270  . 
```

is the BGP to retrieve all **entities that are trademarks**

## Workload Goals

1. Identify the BGP for the company or person related to a given trademark

2. Identify the BGP to retrieve types or categories for a given trademark

3. What companies have the largest number of trademarks? 

4. What are the types of trademarks, how many trademarks exist for each type?

5. Analyze the number of trademarks across types, companies, and countries
 
   5.1 How many U.S. companies hold trademarks? In which sectors?
   
   5.2 How many people hold or are connected to trademarks? In which role?
   
   5.3 In which sector there is the highest number of trademarks?   


#### Example of query

In [2]:
queryString = """
SELECT COUNT(?p) WHERE { 

    ?p wdt:P31/wdt:P279* wd:Q167270  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '44700')]


1

### My Workflow

#### ***Task 1 :*** Identify the BGP for the company or person related to a given trademark

I start showing some examples of entities that are ***trademark (wd:Q167270)***.

In [3]:
queryString = """
SELECT DISTINCT ?trademark ?trademarkName WHERE { 

    # Retrieve entities that are trademarks
    ?trademark wdt:P31/wdt:P279* wd:Q167270  . 

    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .  
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q15874936'), ('trademarkName', 'Michelob')]
[('trademark', 'http://www.wikidata.org/entity/Q2411403'), ('trademarkName', 'Bernard')]
[('trademark', 'http://www.wikidata.org/entity/Q11233843'), ('trademarkName', 'TOP Media')]
[('trademark', 'http://www.wikidata.org/entity/Q17019891'), ('trademarkName', 'Malco Theatres')]
[('trademark', 'http://www.wikidata.org/entity/Q17074818'), ('trademarkName', 'MovieTime Cinemas')]
[('trademark', 'http://www.wikidata.org/entity/Q3527542'), ('trademarkName', 'B2M Entertainment')]
[('trademark', 'http://www.wikidata.org/entity/Q5052147'), ('trademarkName', 'Cathay Organisation')]
[('trademark', 'http://www.wikidata.org/entity/Q17455021'), ('trademarkName', 'Albrecht')]
[('trademark', 'http://www.wikidata.org/entity/Q18161548'), ('trademarkName', 'Network NorthWest')]
[('trademark', 'http://www.wikidata.org/entity/Q21014477'), ('trademarkName', 'H2 Media Entertainment')]
[('trademark', 'http://www.

20

I check if ***iMac (wd:Q14091)*** is a ***trademark (wd:Q167270)*** using an ASK query.

In [4]:
queryString = "ASK {wd:Q14091 wdt:P31/wdt:P279* wd:Q167270}"

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

I have confirmed that  ***iMac (wd:Q14091)*** is a ***trademark (wd:Q167270)***.

I try to retrieve all the properties that connect ***iMac (wd:Q14091)*** to ***Apple Inc. (wd:Q312)***.

In [5]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
             
    # Retrieve all the properties between iMac and Apple Inc.
    wd:Q14091 ?p wd:Q312 .
                
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pName', 'manufacturer')]
[('p', 'http://www.wikidata.org/prop/direct/P178'), ('pName', 'developer')]


2

I use the two discovered properties to connect ***trademark (wd:Q167270)*** to ***Apple Inc.(wd:Q312)***.

In [6]:
queryString = """
SELECT DISTINCT ?trademark ?trademarkName WHERE { 
    
    # Retrieve entities that are trademarks connected to Apple using the properties "developer" and "manufacturer"
    ?trademark  wdt:P31/wdt:P279*     wd:Q167270  ;
                (wdt:P176 | wdt:P178) wd:Q312     .          
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .  
    
} 
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q3065317'), ('trademarkName', 'MacBook')]
[('trademark', 'http://www.wikidata.org/entity/Q14091'), ('trademarkName', 'iMac')]
[('trademark', 'http://www.wikidata.org/entity/Q3683593'), ('trademarkName', 'Color LaserWriter')]
[('trademark', 'http://www.wikidata.org/entity/Q62462617'), ('trademarkName', 'Apple Card')]
[('trademark', 'http://www.wikidata.org/entity/Q697292'), ('trademarkName', 'AirPort')]
[('trademark', 'http://www.wikidata.org/entity/Q18010891'), ('trademarkName', 'iPhone 6 Plus')]
[('trademark', 'http://www.wikidata.org/entity/Q2766'), ('trademarkName', 'iPhone')]
[('trademark', 'http://www.wikidata.org/entity/Q90696274'), ('trademarkName', 'iPhone SE (2nd generation)')]
[('trademark', 'http://www.wikidata.org/entity/Q406668'), ('trademarkName', 'AirPlay')]


9

Since there are very few results, I want to check if there is also a property from ***Apple Inc. (wd:Q312)*** to ***iMac (wd:Q14091)*** (opposite direction from the previous queries).

In [7]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
             
    # I retrieve all the properties between Apple and iMac
    wd:Q312 ?p wd:Q14091.
                
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
    
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]


1

I use the just discovered property ***product or material produced (wdt:P1056)*** to connect ***Apple Inc.(wd:Q312)*** and entities of ***trademark (wd:Q167270)***.

In [8]:
queryString = """
SELECT DISTINCT ?trademark ?trademarkName WHERE { 
    
    # Retrieve entities that are trademarks
    ?trademark  wdt:P31/wdt:P279*  wd:Q167270  .
    
    # Connect Apple to trademarks
    wd:Q312 wdt:P1056 ?trademark .          
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .   
} 
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q18010891'), ('trademarkName', 'iPhone 6 Plus')]
[('trademark', 'http://www.wikidata.org/entity/Q14091'), ('trademarkName', 'iMac')]
[('trademark', 'http://www.wikidata.org/entity/Q90696274'), ('trademarkName', 'iPhone SE (2nd generation)')]


3

The results of this query are already included in the results of the query that makes use of ***manufacturer (wdt:P176)*** and ***developer (wdt:P178)***.

Maybe there is another way to connect a ***trademark (wd:Q167270)*** to companies. I show all the properties that connect a ***trademark (wd:Q167270)*** to ***Apple Inc.(wd:Q312)***.

In [9]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
    
    # Retrieve entities that are trademarks connected to Apple
    ?trademark  wdt:P31/wdt:P279*   wd:Q167270  ;
                ?p                  wd:Q312     .          
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .  
    
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P178'), ('pName', 'developer')]
[('p', 'http://www.wikidata.org/prop/direct/P176'), ('pName', 'manufacturer')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pName', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P749'), ('pName', 'parent organization')]


5

Since I work with trademarks, I have probably to consider also the property ***owned by (wdt:P127)***.

So, I can use this BGP to retrieve a company related to a trademark:
```
?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
            (wdt:P127 | wdt:P176 | wdt:P178)   ?company   . 
```
But how can I retrieve people connected to a trademark ? Maybe these three properties do not connect only companies but also people.

I use ***instanceOf (wdt:P31)*** to see if also humans are connected through this property.

In [10]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner     . 
    
    # Using the property instanceOf to see what is connected
    ?owner wdt:P31 ?class .          
    
    # This returns the labels
    ?class <http://schema.org/name> ?className .  
    
    # I want to search for something that contains the word "human"
    FILTER (REGEX (?className,"human"))  
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5'), ('className', 'human')]


1

So the three discovered properties works also for istances of ***human (wd:Q5)***. Indeed, I can show some examples of trademarks owned by istances of ***human (wd:Q5)***.

In [11]:
queryString = """
SELECT DISTINCT ?trademark ?trademarkName ?ownerName WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner     . 
    
    # I want only human owners
    ?owner wdt:P31  wd:Q5 .          
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?owner     <http://schema.org/name> ?ownerName     .     
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q24908552'), ('trademarkName', '100 Thieves'), ('ownerName', 'Scooter Braun')]
[('trademark', 'http://www.wikidata.org/entity/Q106306786'), ('trademarkName', 'Starboy Entertainment'), ('ownerName', 'Wizkid')]
[('trademark', 'http://www.wikidata.org/entity/Q108031835'), ('trademarkName', 'Davido Music Worldwide'), ('ownerName', 'Davido')]
[('trademark', 'http://www.wikidata.org/entity/Q27056912'), ('trademarkName', 'Record Rebellion'), ('ownerName', 'Christopher James Sampson')]
[('trademark', 'http://www.wikidata.org/entity/Q1150860'), ('trademarkName', 'Cîroc'), ('ownerName', 'Sean Combs')]
[('trademark', 'http://www.wikidata.org/entity/Q3156934'), ('trademarkName', 'JOB'), ('ownerName', 'Pierre Bardou-Job')]
[('trademark', 'http://www.wikidata.org/entity/Q1228353'), ('trademarkName', 'DiscReet'), ('ownerName', 'Frank Zappa')]
[('trademark', 'http://www.wikidata.org/entity/Q17032457'), ('trademarkName', 'Wahlburgers'), ('ownerName

20

So the three discovered properties: (***manufacturer (wdt:P176)***, ***developer (wdt:P178)***, ***owned by (wdt:P127)***) work both for companies and people.

I show some examples using GROUP_CONCAT to group different owners of the same ***trademark (wd:Q167270)***.

In [12]:
queryString = """
SELECT ?trademark ?trademarkName (GROUP_CONCAT(DISTINCT ?ownerName ; separator = ", ") AS ?owners) WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   .         
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?owner <http://schema.org/name>     ?ownerName .  
}
GROUP BY ?trademark ?trademarkName
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q3192162'), ('trademarkName', 'Kalenji'), ('owners', 'Decathlon')]
[('trademark', 'http://www.wikidata.org/entity/Q15853674'), ('trademarkName', 'Wartburg'), ('owners', 'Automobilwerk Eisenach')]
[('trademark', 'http://www.wikidata.org/entity/Q794166'), ('trademarkName', 'Adidas Azteca'), ('owners', 'Adidas')]
[('trademark', 'http://www.wikidata.org/entity/Q3303481'), ('trademarkName', 'Cinnamon Toast Crunch'), ('owners', 'General Mills, Nestlé')]
[('trademark', 'http://www.wikidata.org/entity/Q104858729'), ('trademarkName', 'Hot Damn!'), ('owners', 'DeKuyper')]
[('trademark', 'http://www.wikidata.org/entity/Q67876003'), ('trademarkName', 'YF-Vax'), ('owners', 'Sanofi Pasteur')]
[('trademark', 'http://www.wikidata.org/entity/Q3276886'), ('trademarkName', 'Magic Life'), ('owners', 'TUI AG')]
[('trademark', 'http://www.wikidata.org/entity/Q3021725'), ('trademarkName', 'Dell XPS'), ('owners', 'Dell Inc.')]
[('trademark', 'http://www.w

30

#### END TASK 1:
The properties ***manufacturer (wdt:P176)***, ***developer (wdt:P178)***, ***owned by (wdt:P127)***) work both for companies and people.

Hence, the BGP for the company or person related to a given trademark is:
```
?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
            (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 
```

It is not possible to use only the property ***owned by (wdt:P127)***, because I could not retrieve some connections like the one between ***iMac (wd:Q14091)*** to ***Apple Inc. (wd:Q312)*** (as showed before).

### ***Task 2 :*** Identify the BGP to retrieve types or categories for a given trademark

I start using the property ***subclassOf (wdt:P279)*** on ***iMac (wd:Q14091)***.

In [13]:
queryString = """
SELECT DISTINCT ?subclass ?subclassName WHERE { 
    
    # Retrieve subclasses of iMac
    ?subclass wdt:P279 wd:Q14091 .         
    
    # This returns the labels
    ?subclass <http://schema.org/name> ?subclassName .   
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('subclass', 'http://www.wikidata.org/entity/Q306381'), ('subclassName', 'iMac G3')]
[('subclass', 'http://www.wikidata.org/entity/Q1050381'), ('subclassName', 'iMac G5')]
[('subclass', 'http://www.wikidata.org/entity/Q2407708'), ('subclassName', 'iMac G4')]
[('subclass', 'http://www.wikidata.org/entity/Q106576259'), ('subclassName', 'iMac (Apple silicon)')]


4

I try to apply the same technique to all the instances of ***trademark (wd:Q167270)***. I use ***GROUP_CONCAT*** to list all the types of a ***trademark (wd:Q167270)***.

In [14]:
queryString = """
SELECT ?trademark ?trademarkName (GROUP_CONCAT(DISTINCT ?subclassName ; separator = ", ") AS ?types) WHERE { 
    
    # Retrieve entities that are trademarks 
    ?trademark  wdt:P31/wdt:P279* wd:Q167270 .       
    
    # Retrieve subclasses of trademarks
    ?subclass wdt:P279 ?trademark . 
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?subclass  <http://schema.org/name> ?subclassName .  
}
GROUP BY ?trademark ?trademarkName
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q386903'), ('trademarkName', 'Intel Core i7'), ('types', 'Core i7 980X, Core i7-4770HQ, Core i7-6700, Core i7-7700, Core i7-7700K, Core i7-7700T')]
[('trademark', 'http://www.wikidata.org/entity/Q623561'), ('trademarkName', 'Dr Pepper'), ('types', 'Diet Dr Pepper')]
[('trademark', 'http://www.wikidata.org/entity/Q129337'), ('trademarkName', 'TGV'), ('types', 'SNCF TGV La Poste, TERGV, TGV inOui')]
[('trademark', 'http://www.wikidata.org/entity/Q73389'), ('trademarkName', 'Samsung Galaxy S series'), ('types', 'Samsung Galaxy Tab S3')]
[('trademark', 'http://www.wikidata.org/entity/Q1123613'), ('trademarkName', 'Schuko'), ('types', 'splashproof Schuko')]
[('trademark', 'http://www.wikidata.org/entity/Q131535'), ('trademarkName', 'Ariane'), ('types', 'Ariane 1, Ariane 2, Ariane 3, Ariane 4, Ariane 5, Ariane 6')]
[('trademark', 'http://www.wikidata.org/entity/Q105336048'), ('trademarkName', 'CYCLO Banh mi'), ('types', 'CYCLO Banh mi(Eb

15

It works as expected.

##### END TASK 2 
The BGP to retrieve types or categories for a given trademark is:
```
?trademark  wdt:P31/wdt:P279* wd:Q167270 .       
?subclass   wdt:P279          ?trademark . 
```

### ***Task 3 :*** What companies have the largest number of trademarks?

I can use the same BGP of Task 1, but since I want only companies I have to exclude instances of ***trademark (wd:Q167270)*** owned by instances of ***human (wd:Q5)***

In [15]:
queryString = """
SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner     . 

    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?owner     <http://schema.org/name> ?ownerName     .  
    
    # I want to exclude trademarks owned by humans 
    FILTER NOT EXISTS { ?owner wdt:P31  wd:Q5 . }   
}
GROUP BY ?owner ?ownerName
ORDER BY DESC(?numTrademarks)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('owner', 'http://www.wikidata.org/entity/Q160746'), ('ownerName', 'Nestlé'), ('numTrademarks', '77')]
[('owner', 'http://www.wikidata.org/entity/Q3295867'), ('ownerName', 'The Coca-Cola Company'), ('numTrademarks', '72')]
[('owner', 'http://www.wikidata.org/entity/Q812142'), ('ownerName', 'Covestro'), ('numTrademarks', '57')]
[('owner', 'http://www.wikidata.org/entity/Q334800'), ('ownerName', 'PepsiCo'), ('numTrademarks', '48')]
[('owner', 'http://www.wikidata.org/entity/Q6677525'), ('ownerName', 'Cadbury'), ('numTrademarks', '36')]
[('owner', 'http://www.wikidata.org/entity/Q157062'), ('ownerName', 'Unilever'), ('numTrademarks', '35')]
[('owner', 'http://www.wikidata.org/entity/Q12857502'), ('ownerName', 'Mondelēz International'), ('numTrademarks', '34')]
[('owner', 'http://www.wikidata.org/entity/Q1056637'), ('ownerName', 'The Hershey Company'), ('numTrademarks', '32')]
[('owner', 'http://www.wikidata.org/entity/Q856897'), ('ownerName', "Kellogg's"), ('numTrademarks', '31')

10

##### END TASK 3
I was able to show the companies with the largest number of trademarks. In particular, ***Nestlé (wd:Q160746)*** has the largest one (77 trdemarks) .

### ***Task 4 :*** What are the types of trademarks, how many trademarks exist for each type?

The task is quite similar to Task 2, but instead of searching for sub-types of a given trademark, I use the property ***subclassOf (wdt:P279)*** to search for different types of ***trademark (wd:Q167270)***. 

I show some examples.

In [16]:
queryString = """
SELECT ?trademark ?trademarkName (GROUP_CONCAT(DISTINCT ?className ; separator = ", ") AS ?types) WHERE { 
    
    # Retrieve entities that are trademarks 
    ?trademark  wdt:P31/wdt:P279* wd:Q167270 .       
    
    # Retrieve classes of trademarks
    ?trademark  wdt:P279 ?class . 
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?class     <http://schema.org/name> ?className    .  
}
GROUP BY ?trademark ?trademarkName
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('trademark', 'http://www.wikidata.org/entity/Q3304193'), ('trademarkName', 'Selsun Blue'), ('types', 'shampoo')]
[('trademark', 'http://www.wikidata.org/entity/Q17766439'), ('trademarkName', 'Diet Mountain Dew'), ('types', 'diet soda')]
[('trademark', 'http://www.wikidata.org/entity/Q932080'), ('trademarkName', 'Kvikk Lunsj'), ('types', 'chocolate bar')]
[('trademark', 'http://www.wikidata.org/entity/Q99613456'), ('trademarkName', 'Supplyco Super Market'), ('types', 'outlet store')]
[('trademark', 'http://www.wikidata.org/entity/Q4944122'), ('trademarkName', 'Onyx Boox'), ('types', 'e-book reader')]
[('trademark', 'http://www.wikidata.org/entity/Q1050530'), ('trademarkName', 'Tic Tac'), ('types', 'confection, mint')]
[('trademark', 'http://www.wikidata.org/entity/Q5400907'), ('trademarkName', 'Presidente'), ('types', 'beer')]
[('trademark', 'http://www.wikidata.org/entity/Q1342742'), ('trademarkName', 'RC Cola'), ('types', 'cola')]
[('trademark', 'http://www.wikidata.org/enti

10

I show some examples of different types of ***trademark (wd:Q167270)***. 

In [17]:
queryString = """
SELECT DISTINCT ?className WHERE { 
    
    # Retrieve entities that are trademarks 
    ?trademark  wdt:P31/wdt:P279* wd:Q167270 .       
    
    # Retrieve classes of trademarks
    ?trademark  wdt:P279 ?class . 
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?class     <http://schema.org/name> ?className     .  
}
GROUP BY ?trademark ?trademarkName
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('className', 'lager')]
[('className', 'abbey beer')]
[('className', 'grocery store')]
[('className', 'tire shop')]
[('className', 'bakery')]
[('className', 'restaurant')]
[('className', 'ramen shop')]
[('className', 'candy bar')]
[('className', 'vodka')]
[('className', 'drink mix')]


10

I can finally count how many istances of ***trademark (wd:Q167270)*** exist for each type.

In [18]:
queryString = """
SELECT ?class ?className COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 
    
    # Retrieve entities that are trademarks 
    ?trademark  wdt:P31/wdt:P279* wd:Q167270 .       
    
    # Retrieve classes of trademarks
    ?trademark  wdt:P279 ?class . 
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?class  <http://schema.org/name> ?className .  
}
GROUP BY ?class ?className
ORDER BY DESC(?numTrademarks)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q1361086'), ('className', 'chocolate bar'), ('numTrademarks', '75')]
[('class', 'http://www.wikidata.org/entity/Q11312555'), ('className', 'candy bar'), ('numTrademarks', '67')]
[('class', 'http://www.wikidata.org/entity/Q5159627'), ('className', 'confection'), ('numTrademarks', '47')]
[('class', 'http://www.wikidata.org/entity/Q147538'), ('className', 'soft drink'), ('numTrademarks', '45')]
[('class', 'http://www.wikidata.org/entity/Q22645'), ('className', 'smartphone'), ('numTrademarks', '43')]
[('class', 'http://www.wikidata.org/entity/Q185583'), ('className', 'candy'), ('numTrademarks', '39')]
[('class', 'http://www.wikidata.org/entity/Q768267'), ('className', 'breakfast cereal'), ('numTrademarks', '36')]
[('class', 'http://www.wikidata.org/entity/Q40050'), ('className', 'drink'), ('numTrademarks', '30')]
[('class', 'http://www.wikidata.org/entity/Q374'), ('className', 'vodka'), ('numTrademarks', '29')]
[('class', 'http://www.wikid

10

##### END TASK 4
As I showed, it is possibile to retrieve the different types of trademarks and to count how many trademarks exist for each type.

It is also possible to notice that in the top 10 of types with the largest number of trademarks there are a lot of food products.

### ***Task 5.1 :*** How many U.S. companies hold trademarks? In which sectors?

First of all I need to understand how a company is related to a country. So, I show all the object properties of ***Apple Inc.(wd:Q312)***.

In [19]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
    
    # Connect Apple to something
    wd:Q312 ?p ?o .          
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
    
    # Exclude data properties
    FILTER(!isLiteral(?o))
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pName', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('pName', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('pName', 'chief executive office

20

I can probably use ***country (wdt:P17)***. I try it on ***Apple Inc.(wd:Q312)***.

In [20]:
queryString = """
SELECT DISTINCT ?country ?countryName WHERE { 
    
    # Retrieve Apple's country
    wd:Q312 wdt:P17 ?country.          
    
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
}
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America')]


1

I can count how many companies located in ***United States of America(wd:Q30)*** hold a ***trademark (wd:Q167270)*** .

In [21]:
queryString = """
SELECT COUNT(DISTINCT ?company) AS ?numCompanies WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?company  . 
    
    # I want only american companies
    ?company wdt:P17 wd:Q30 . 
    
    # I exclude trademarks owned by humans because I want only companies
    FILTER NOT EXISTS { ?company wdt:P31  wd:Q5 . }   
        
}
"""

print("Results")
run_query(queryString)

Results
[('numCompanies', '370')]


1

Now, to answer the question "In which sectors?", I have to understand how to connect a company to a specicifc sector.

I show everything that is connected to ***Apple Inc.(wd:Q312)***.

In [22]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName WHERE { 
    
    # Connect Apple to something
    wd:Q312 ?p ?o .          
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
    ?o <http://schema.org/name> ?oName .
    
    # exclude data properties
    FILTER(!isLiteral(?o))
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q26833946'), ('oName', 'iPhone\xa07 Plus')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q39510168'), ('oName', 'iPhone X')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q20962515'), ('oName', 'Apple Pencil')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('pName', 'industry'), ('o', 'http://www.wikidata.org/entity/Q581105'), ('oName', 'consumer electronics')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('o', 'http://www.wikidata.org/entity/Q79100838'), ('oName', 'Zigbee Alliance')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pName', 'product or material produced'), ('o', 'http://www.wikidata.org/entity/Q18010891'), ('oName

20

Maybe I can use the property ***industry (wdt:P452)***. I try it on ***Apple Inc.(wd:Q312)***.

In [23]:
queryString = """
SELECT DISTINCT ?sector ?sectorName WHERE { 
    
    # Connect Apple using the property "industry"
    wd:Q312 wdt:P452 ?sector .          
    
    # This returns the labels
    ?sector <http://schema.org/name> ?sectorName .
}
"""

print("Results")
run_query(queryString)

Results
[('sector', 'http://www.wikidata.org/entity/Q581105'), ('sectorName', 'consumer electronics')]
[('sector', 'http://www.wikidata.org/entity/Q11650'), ('sectorName', 'electronics')]
[('sector', 'http://www.wikidata.org/entity/Q56598901'), ('sectorName', 'mobile phone industry')]
[('sector', 'http://www.wikidata.org/entity/Q11661'), ('sectorName', 'information technology')]
[('sector', 'http://www.wikidata.org/entity/Q880371'), ('sectorName', 'software industry')]
[('sector', 'http://www.wikidata.org/entity/Q269415'), ('sectorName', 'digital distribution')]


6

So, the property ***industry (wdt:P452)*** can be used to retrieve the sectors. I can finally answer to the initial question.

In [24]:
queryString = """
SELECT ?sector ?sectorName COUNT(DISTINCT ?company) AS ?numCompanies WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?company  . 
    
    # Retrieve sectors of american companies
    ?company wdt:P17  wd:Q30  ;
             wdt:P452 ?sector . 
    
    # This returns the labels
    ?sector <http://schema.org/name> ?sectorName .
    
    # I exclude trademarks owned by humans because I want only companies
    FILTER NOT EXISTS { ?company wdt:P31  wd:Q5 . }  
}
GROUP BY ?sector ?sectorName
ORDER BY DESC(?numCompanies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('sector', 'http://www.wikidata.org/entity/Q126793'), ('sectorName', 'retail'), ('numCompanies', '26')]
[('sector', 'http://www.wikidata.org/entity/Q540912'), ('sectorName', 'food industry'), ('numCompanies', '20')]
[('sector', 'http://www.wikidata.org/entity/Q10302058'), ('sectorName', 'phonographic industry'), ('numCompanies', '9')]
[('sector', 'http://www.wikidata.org/entity/Q1187656'), ('sectorName', 'fast-moving consumer goods'), ('numCompanies', '9')]
[('sector', 'http://www.wikidata.org/entity/Q190117'), ('sectorName', 'automotive industry'), ('numCompanies', '8')]
[('sector', 'http://www.wikidata.org/entity/Q746359'), ('sectorName', 'music industry'), ('numCompanies', '8')]
[('sector', 'http://www.wikidata.org/entity/Q4899370'), ('sectorName', 'beverage industry'), ('numCompanies', '8')]
[('sector', 'http://www.wikidata.org/entity/Q25245117'), ('sectorName', 'telecommunications industry'), ('numCompanies', '8')]
[('sector', 'http://www.wikidata.org/entity/Q11661'), ('s

10

##### END TASK 5.1
I discovered that in the ***United States of America(wd:Q30)*** there are 370 companies that holds trademarks.

Moreover, I showed that there are a lot of different sectors in which these companies work such as ***retail, food industry, etc...***

### ***Task 5.2 :*** How many people hold or are connected to trademarks? In which role?

I try to connect entities of ***human(wd:Q5)*** to istances of ***trademark (wd:Q167270)***.

I show some examples.

In [25]:
queryString = """
SELECT DISTINCT ?trademarkName ?pName ?personName WHERE { 
    
    # Retrieve entities that are trademarks and connect them to people
    ?trademark  wdt:P31/wdt:P279*  wd:Q167270 ;
                ?p                 ?person    . 
    
    # I want only people
    ?person wdt:P31 wd:Q5 .  
    
    # This returns the labels
    ?p         <http://schema.org/name> ?pName .
    ?person    <http://schema.org/name> ?personName .
    ?trademark <http://schema.org/name> ?trademarkName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('trademarkName', 'Pathé'), ('pName', 'director / manager'), ('personName', 'Jérôme Seydoux')]
[('trademarkName', 'Orange Sky Golden Harvest'), ('pName', 'founded by'), ('personName', 'Leonard Ho')]
[('trademarkName', 'Orange Sky Golden Harvest'), ('pName', 'founded by'), ('personName', 'Raymond Chow')]
[('trademarkName', 'YG Entertainment'), ('pName', 'founded by'), ('personName', 'Yang Hyun-seok')]
[('trademarkName', 'Pathé'), ('pName', 'founded by'), ('personName', 'Charles Pathé')]
[('trademarkName', 'MK2'), ('pName', 'founded by'), ('personName', 'Marin Karmitz')]
[('trademarkName', 'Huayi Brothers'), ('pName', 'founded by'), ('personName', 'Wang Zhongjun')]
[('trademarkName', 'Hybe Corporation'), ('pName', 'founded by'), ('personName', 'Bang Si-hyuk')]
[('trademarkName', 'Orange Sky Golden Harvest'), ('pName', 'founded by'), ('personName', 'Leung Fung')]
[('trademarkName', 'TOP Media'), ('pName', 'founded by'), ('personName', 'Andy Lee')]


10

I show only the properties that connect istances of ***trademark (wd:Q167270)*** to people.

In [26]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 
    
    # Retrieve entities that are trademarks and connect them to people
    ?trademark  wdt:P31/wdt:P279*  wd:Q167270 ;
                ?p                 ?person    . 
    
    # I want only people
    ?person wdt:P31 wd:Q5 .  
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pName', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('pName', 'chief executive officer')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pName', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P3931'), ('pName', 'copyright holder')]


10

Now, I can count how many people are related to trademarks grouping them by the property used.

In [27]:
queryString = """
SELECT ?p ?pName COUNT(DISTINCT ?person) as ?numPeople WHERE { 
    
    # Retrieve entities that are trademarks and connect them to people
    ?trademark  wdt:P31/wdt:P279*  wd:Q167270 ;
                ?p                 ?person    . 
    
    # I want only people
    ?person wdt:P31 wd:Q5 .  
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
GROUP BY ?p ?pName
ORDER BY DESC(?numPeople)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by'), ('numPeople', '2424')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by'), ('numPeople', '236')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('numPeople', '161')]
[('p', 'http://www.wikidata.org/prop/direct/P169'), ('pName', 'chief executive officer'), ('numPeople', '150')]
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pName', 'director / manager'), ('numPeople', '86')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('numPeople', '51')]
[('p', 'http://www.wikidata.org/prop/direct/P61'), ('pName', 'discoverer or inventor'), ('numPeople', '51')]
[('p', 'http://www.wikidata.org/prop/direct/P3320'), ('pName', 'board member'), ('numPeople', '49')]
[('p', 'http://www.wikidata.org/prop/direct/P488'), ('pName', 'chairperson'), ('numPeople', '41')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pName', 'creator'

10

##### END TASK 5.2
It is possible to notice that people are connected to istances of ***trademark (wd:Q167270)*** not only by the property ***owned by (wdt:P127)*** but also by other properties.

In particular there are 2424 people connected through the proerty ***founded by (wdt:P112)***. This is probably due to the fact that a lot of istances of ***trademark (wd:Q167270)*** are companies/brands and so there is at least one founder for each of them.

### ***Task 5.3 :*** In which sector there is the highest number of trademarks?

The query is similar to the one used in Task 5.1, but instead of counting how many US companies I have for each sector, I count how many istances of ***trademark (wd:Q167270)*** there are.

In [28]:
queryString = """
SELECT ?sector ?sectorName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?company  . 
    
    # Retrieve sectors of companies
    ?company wdt:P452 ?sector . 
    
    # This returns the labels
    ?sector <http://schema.org/name> ?sectorName .
    
    # I exclude trademarks owned by humans because I want only companies
    FILTER NOT EXISTS { ?company wdt:P31  wd:Q5 . }  
}
GROUP BY ?sector ?sectorName
ORDER BY DESC(?numTrademarks)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('sector', 'http://www.wikidata.org/entity/Q540912'), ('sectorName', 'food industry'), ('numTrademarks', '583')]
[('sector', 'http://www.wikidata.org/entity/Q869095'), ('sectorName', 'brewing'), ('numTrademarks', '516')]
[('sector', 'http://www.wikidata.org/entity/Q1187656'), ('sectorName', 'fast-moving consumer goods'), ('numTrademarks', '309')]
[('sector', 'http://www.wikidata.org/entity/Q4899370'), ('sectorName', 'beverage industry'), ('numTrademarks', '263')]
[('sector', 'http://www.wikidata.org/entity/Q107601756'), ('sectorName', 'food and tobacco industry'), ('numTrademarks', '223')]
[('sector', 'http://www.wikidata.org/entity/Q126793'), ('sectorName', 'retail'), ('numTrademarks', '192')]
[('sector', 'http://www.wikidata.org/entity/Q207652'), ('sectorName', 'chemical industry'), ('numTrademarks', '175')]
[('sector', 'http://www.wikidata.org/entity/Q190117'), ('sectorName', 'automotive industry'), ('numTrademarks', '144')]
[('sector', 'http://www.wikidata.org/entity/Q7463

10

##### END TASK 5.3
The ***food industry (wd:Q540912)*** is the sector with the highest number of trademarks (583 trademarks)

### Extra Analytics query

#### On average how many trademarks are owned by a single company ?

In [29]:
queryString = """
SELECT AVG(?numTrademarks) AS ?avgNumTrademarks{

    {   SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 

            # Retrieve entities that are trademarks and connect them to their "owner"
            ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                        (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 

            # This returns the labels
            ?trademark <http://schema.org/name> ?trademarkName .
            ?owner <http://schema.org/name>     ?ownerName .  

            # I want to exclude trademarks owned by humans 
            FILTER NOT EXISTS { ?owner wdt:P31  wd:Q5 . }   
        }
        GROUP BY ?owner ?ownerName
    }
}
"""

print("Results")
run_query(queryString)

Results
[('avgNumTrademarks', '1.98361581920904')]


1

#### On average how many trademarks are owned by a US company ?

In [30]:
queryString = """
SELECT AVG(?numTrademarks) AS ?avgNumTrademarks{

    {   SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 

            # Retrieve entities that are trademarks and connect them to their "owner"
            ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                        (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 

            # I want only american companies
            ?owner wdt:P17 wd:Q30 . 
            
            # This returns the labels
            ?trademark <http://schema.org/name> ?trademarkName .
            ?owner <http://schema.org/name>     ?ownerName .  

            # I want to exclude trademarks owned by humans 
            FILTER NOT EXISTS { ?owner wdt:P31  wd:Q5 . }   
        }
        GROUP BY ?owner ?ownerName
    }
}
"""

print("Results")
run_query(queryString)

Results
[('avgNumTrademarks', '3.10655737704918')]


1

#### How many trademarks are owned by companies grouped by country ?

In [31]:
queryString = """
SELECT ?country ?countryName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 
    
    # Retrieve the country of a company
    ?owner wdt:P17 ?country . 
            
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?owner     <http://schema.org/name> ?ownerName . 
    ?country   <http://schema.org/name> ?countryName .
    
    # I want to exclude trademarks owned by humans 
    FILTER NOT EXISTS { ?owner wdt:P31  wd:Q5 . }   
}
GROUP BY ?country ?countryName
ORDER BY DESC(?numTrademarks)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('numTrademarks', '1085')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numTrademarks', '308')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('numTrademarks', '241')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan'), ('numTrademarks', '193')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numTrademarks', '147')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy'), ('numTrademarks', '101')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium'), ('numTrademarks', '97')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('numTrademarks', '96')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('numTrademarks', '81')]
[('country

10

#### On average how many trademarks are owned by a single company for each country?

In [32]:
queryString = """
SELECT ?country ?countryName AVG(?numTrademarks) AS ?avgNumTrademarksByCountry{

    # I retrieve the country of a company
    ?owner wdt:P17 ?country . 
    
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .  
            
    {   SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 

            # Retrieve entities that are trademarks and connect them to their "owner"
            ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                        (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 
            
            # This returns the labels
            ?trademark <http://schema.org/name> ?trademarkName .
            ?owner <http://schema.org/name>     ?ownerName .  

            # I want to exclude trademarks owned by humans 
            FILTER NOT EXISTS { ?owner wdt:P31  wd:Q5 . }   
        }
        GROUP BY ?owner ?ownerName
    }   
}
GROUP BY ?country ?countryName
ORDER BY DESC(?avgNumTrademarksByCountry)
LIMIT 10 
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q174193'), ('countryName', 'United Kingdom of Great Britain and Ireland'), ('avgNumTrademarksByCountry', '36')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('avgNumTrademarksByCountry', '5.647058823529412')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryName', 'Netherlands'), ('avgNumTrademarksByCountry', '3.375')]
[('country', 'http://www.wikidata.org/entity/Q41'), ('countryName', 'Greece'), ('avgNumTrademarksByCountry', '3.333333333333333')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('avgNumTrademarksByCountry', '3.10655737704918')]
[('country', 'http://www.wikidata.org/entity/Q25230'), ('countryName', 'Guernsey'), ('avgNumTrademarksByCountry', '3')]
[('country', 'http://www.wikidata.org/entity/Q785'), ('countryName', 'Jersey'), ('avgNumTrademarksByCountry', '3')]
[('country', 'http://www.wikidata.org/entity/Q884'), ('count

10

#### Who are the people that own the highest number of trademarks ?

In [33]:
queryString = """
SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 
    
    # Retrieve entities that are trademarks and connect them to their "owner"
    ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 

    # I want only trademarks owned by humans 
    ?owner wdt:P31  wd:Q5 . 
    
    # This returns the labels
    ?trademark <http://schema.org/name> ?trademarkName .
    ?owner <http://schema.org/name>     ?ownerName .  
    
}
GROUP BY ?owner ?ownerName
ORDER BY DESC(?numTrademarks)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('owner', 'http://www.wikidata.org/entity/Q30201990'), ('ownerName', 'Pedro Fernando'), ('numTrademarks', '6')]
[('owner', 'http://www.wikidata.org/entity/Q36844'), ('ownerName', 'Rihanna'), ('numTrademarks', '3')]
[('owner', 'http://www.wikidata.org/entity/Q909017'), ('ownerName', 'Brennan Heart'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q3170637'), ('ownerName', 'Jean Baud'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q105455093'), ('ownerName', 'Sadoc Vazkez'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q6524850'), ('ownerName', 'Leon René'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q77428188'), ('ownerName', 'Gundolf Meyer-Hentschel'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q7781479'), ('ownerName', 'Theo Paphitis'), ('numTrademarks', '2')]
[('owner', 'http://www.wikidata.org/entity/Q106893140'), ('ownerName', 'Mina'), ('numTrademarks', '2')]
[('

10

#### On average how many trademarks are owned by a single person ? 

In [34]:
queryString = """
SELECT AVG(?numTrademarks) AS ?avgNumTrademarks{

    {   SELECT ?owner ?ownerName COUNT(DISTINCT ?trademark) AS ?numTrademarks WHERE { 

            # Retrieve entities that are trademarks and connect them to their "owner"
            ?trademark  wdt:P31/wdt:P279*                  wd:Q167270 ;
                        (wdt:P127 | wdt:P176 | wdt:P178)   ?owner   . 

            # I want only trademarks owned by humans
            ?owner wdt:P31  wd:Q5 . 
            
            # This returns the labels
            ?trademark <http://schema.org/name> ?trademarkName .
            ?owner <http://schema.org/name>     ?ownerName .  
        }
        GROUP BY ?owner ?ownerName
    }
}
"""

print("Results")
run_query(queryString)

Results
[('avgNumTrademarks', '1.083333333333333')]


1

## END